In [2]:
import pandas as pd
import re
#import pymorphy2
!pip install spacy
import spacy
#!spacy download en

# Чтение данных
Сет распределен по нескольким файлам, поэтому необходимо их объединить.
Сет содержит данные о сложности вопроса по мнению спрашивающего и отвечающего и номер статьи, из которой взят ответ - нам это не нужно

In [3]:
answers1 = pd.read_csv('S08_question_answer_pairs.txt', sep='\t')
answers2 = pd.read_csv('S09_question_answer_pairs.txt', sep='\t')
answers3 = pd.read_csv('S10_question_answer_pairs.txt', sep='\t', encoding='latin-1')

answers = pd.concat([answers1, answers2, answers3])
answers.drop(labels=['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis=1, inplace=True)
answers.head(20)

,ArticleTitle,Question,Answer
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Abraham_Lincoln,Did his mother die of pneumonia?,no
5,Abraham_Lincoln,Did his mother die of pneumonia?,No.
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months
7,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months.
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832
9,Abraham_Lincoln,When did Lincoln begin his political career?,1832.


In [4]:
print('Наличие null:', answers.isnull().values.any())
print('Наличие NaN:', answers.isna().values.any())

Наличие null: True
Наличие NaN: True


# Первичная обработка данных
## Обработка дубликатов и пустых значений

1. Сет содержит вопросы без ответа
2. Сет содержит повторяющиеся вопросы 

Дубликаты неоходимо удалить - они не несут дополнительной смысловой нагрузки, но увеличивают объем для обработки.
Пары с пустыми значениями (без ответа) не могут принеси пользы для модели + нельзя выполнить замену для пустых начени, поскольку необходимо знать правилный ответ. В результате остается только удалить такие вопросы.

Сначала удаляем вопросы без ответа, затем повторяющиеся вопросы

In [5]:
print('Размер начального сета', answers.size)

#предполагается, что нам необходимо заполнение всех полей
not_null_answers = answers.dropna(axis=0, how='any')
print('Размер сета без пустых значений', not_null_answers.size)

# при удалении дупликатов оставляем только первое вхождение
no_duplicates_answers = not_null_answers.drop_duplicates(subset='Question', keep='first')
print('Размер сета без дубликатов', no_duplicates_answers.size)
no_duplicates_answers.head(20)


Размер начального сета 11991
Размер сета без пустых значений 10263
Размер сета без дубликатов 6606


,ArticleTitle,Question,Answer
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes
4,Abraham_Lincoln,Did his mother die of pneumonia?,no
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832
10,Abraham_Lincoln,What did The Legal Tender Act of 1862 establish?,"the United States Note, the first paper curren..."
12,Abraham_Lincoln,Who suggested Lincoln grow a beard?,11-year-old Grace Bedell
14,Abraham_Lincoln,When did the Gettysburg address argue that Ame...,1776
16,Abraham_Lincoln,Did Lincoln beat John C. Breckinridge in the 1...,yes
18,Abraham_Lincoln,Was Abraham Lincoln the first President of the...,No


In [6]:
print('Наличие null:', no_duplicates_answers.isnull().values.any())
print('Наличие NaN:', no_duplicates_answers.isna().values.any())

Наличие null: False
Наличие NaN: False


## Обработка притяжательных местоимений

1. В сете содержатся указания на одушевленный предмет статьи -his/her, he/she 
#####    (напр. Was his (Alessandro_Volta) 1800 paper written in French ?)
2. В сете содержатся указания на неодушевленный предмет статьи -this
#####    (напр. 	What connected the Akans to this (Ghana) Empire?)

### A) Замена на тематику вопроса

Производится замена местоимений на тему, однако только в тех случаях, когда тема не содержится в вопросе и это местоимение является явным на неё указателем: 
##### (напр. "When did Lincoln begin his political career?").
В данном случае замена не производится.

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

#no_duplicates_answers['Question'] =no_duplicates_answers['Question'].replace(regex=[r' his | her | this '], 
#                                                                           value=' замена ')
exp = no_duplicates_answers.copy()
for index, row in exp.iterrows():
    row['ArticleTitle'] = row['ArticleTitle'].replace('_', ' ')
    
    #title_words = list(map(lambda x: x.lower(), row['ArticleTitle'].split()))
    title_words = row['ArticleTitle'].split()
    
    #question_words = re.findall(r"[a-zA-Z]+-[a-zA-Z]+|[a-zA-Z]+\'[a-zA-Z]+|[a-zA-Z]+", row['Question'])
    
    doc = nlp(row['Question'])
    infinitives = [token.lemma_ for token in doc]
    '''
    infinitives = []
    morph = pymorphy2.MorphAnalyzer()
    for word in question_words:
        word = word.lower()
        infinitives.append(morph.parse(word)[0].normal_form)
    '''    
    has_title_in_question = any(map(lambda x: x in infinitives, title_words))
    
    if (not has_title_in_question):
        print(title_words)
        print(infinitives)
        print(row['ArticleTitle'], row['Question'])

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [176]:
no_duplicates_answers


,ArticleTitle,Question,Answer
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes
4,Abraham_Lincoln,Did замена mother die of pneumonia?,no
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months
8,Abraham_Lincoln,When did Lincoln begin замена political career?,1832
10,Abraham_Lincoln,What did The Legal Tender Act of 1862 establish?,"the United States Note, the first paper curren..."
12,Abraham_Lincoln,Who suggested Lincoln grow a beard?,11-year-old Grace Bedell
14,Abraham_Lincoln,When did the Gettysburg address argue that Ame...,1776
16,Abraham_Lincoln,Did Lincoln beat John C. Breckinridge in the 1...,yes
18,Abraham_Lincoln,Was Abraham Lincoln the first President of the...,No
